# Setup imports and loader

In [2]:
import matplotlib.pyplot as plt
import incense
from common.helper.config import parseMongoConfig

loader = incense.ExperimentLoader(**parseMongoConfig('../config.ini', adapt='IncenseExperimentLoader'))

## Find failed Experiments, due to misconfiguration
The problem was, that pytorch does not support PerChannel activation quantization.

In [6]:
from collections import defaultdict


failed_query = {
    '$and': [
        {
            'experiment.name': 'PTSQ_CNN-IMU_LARa'
        },
        {
            'status': 'FAILED'
        },
        {
            '_id': {'$gte': 240}
        },
    ]
}
failed_experiments = loader.find(failed_query)

print(f'Found {len(failed_experiments)} failed Experiments')

exs_by_relevant_config_tuples = defaultdict(list)
for ex in failed_experiments:
  cfg = ex.to_dict()['config']
  exs_by_relevant_config_tuples[(cfg['activation_observer'], cfg['weight_observer'])].append(ex)

for (activation, weight), exs in exs_by_relevant_config_tuples.items():
  print(f'- activation = {activation}')
  print(f'  weight     = {weight}')
  for trace in set([ex.to_dict()["fail_trace"][-1] for ex in exs]):
    print(f'    {trace}',end='')

Found 180 failed Experiments
- activation = torch.ao.quantization.PerChannelMinMaxObserver
  weight     = torch.ao.quantization.MinMaxObserver
    ValueError: only one element tensors can be converted to Python scalars
    RuntimeError: The size of tensor a (17) must match the size of tensor b (32) at non-singleton dimension 0
- activation = torch.ao.quantization.PerChannelMinMaxObserver
  weight     = torch.ao.quantization.PerChannelMinMaxObserver
    ValueError: only one element tensors can be converted to Python scalars
    RuntimeError: The size of tensor a (17) must match the size of tensor b (32) at non-singleton dimension 0
- activation = torch.ao.quantization.MovingAveragePerChannelMinMaxObserver
  weight     = torch.ao.quantization.MinMaxObserver
    ValueError: only one element tensors can be converted to Python scalars
    RuntimeError: The size of tensor a (17) must match the size of tensor b (32) at non-singleton dimension 0
- activation = torch.ao.quantization.MovingAvera

## Confirm by checking all `COMPLETED` experiments
Exactly all PerTensor activation observers succeeded

In [22]:
completed_query = {
    '$and': [
        {
            'experiment.name': 'PTSQ_CNN-IMU_LARa'
        },
        {
            'status': 'COMPLETED'
        },
        {
            '_id': {
                '$gte': 241
            }
        },
    ]
}
completed_experiments = loader.find(completed_query)
print(f'There are {len(completed_experiments)} completed experiments')
observer_cfgs = [(e.to_dict()['config']['activation_observer'],
                  e.to_dict()['config']['weight_observer']) for e in completed_experiments]

print('Distinct observer combinations:')
for act, wgt in set(observer_cfgs):
  print(f'- activation = {act}')
  print(f'  weight     = {wgt}')


There are 270 completed experiments
Distinct observer combinations:
- activation = torch.ao.quantization.MovingAverageMinMaxObserver
  weight     = torch.ao.quantization.PerChannelMinMaxObserver
- activation = torch.ao.quantization.MovingAverageMinMaxObserver
  weight     = torch.ao.quantization.MinMaxObserver
- activation = torch.ao.quantization.MinMaxObserver
  weight     = torch.ao.quantization.PerChannelMinMaxObserver
- activation = torch.ao.quantization.MinMaxObserver
  weight     = torch.ao.quantization.MinMaxObserver
- activation = torch.ao.quantization.HistogramObserver
  weight     = torch.ao.quantization.PerChannelMinMaxObserver
- activation = torch.ao.quantization.HistogramObserver
  weight     = torch.ao.quantization.MinMaxObserver
